In [50]:
import numpy as np
from PIL import Image
import os
import string
from pickle import dump
from pickle import load
from keras._tf_keras.keras.applications.xception import Xception #to get pre-trained model Xception
from keras._tf_keras.keras.applications.xception import preprocess_input
from keras._tf_keras.keras.preprocessing.image import load_img
from keras._tf_keras.keras.preprocessing.image import img_to_array
from keras._tf_keras.keras.preprocessing.text import Tokenizer #for text tokenization
from keras._tf_keras.keras.preprocessing.sequence import pad_sequences
from keras._tf_keras.keras.utils import to_categorical
from keras._tf_keras.keras.models import Model, load_model
from keras._tf_keras.keras.layers import Input, Dense#Keras to build our CNN and LSTM
from keras._tf_keras.keras.layers import LSTM, Embedding, Dropout
from tqdm.notebook import tqdm as tqdm


In [51]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def img_capt(filename):
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split(",")
        
        # take the first token as image id, the rest as description
        image_id, image_desc = tokens[0], tokens[1:]
        
        # extract filename from image id
        image_id = image_id.split('.')[0]
        
        # convert description tokens back to string
        image_desc = ' '.join(image_desc)
        if image_id not in descriptions:
            descriptions[image_id] = list()
        descriptions[image_id].append(image_desc)
    
    return descriptions


In [52]:
def txt_clean(captions):
    table = str.maketrans('','',string.punctuation)
    for img, caps in captions.items():
        for i in range(len(caps)):
            descp = caps[i]
            descp = descp.split()
            #uppercase to lowercase
            descp = [word.lower() for word in descp]
            #remove punctuation from each token
            descp = [word.translate(table) for word in descp]
            #remove hanging 's and a
            descp = [word for word in descp if(len(word)>1)]
            #remove words containing numbers with them
            descp = [word for word in descp if(word.isalpha())]
            #converting back to string
            caps[i] = ' '.join(descp)
    
    return captions

In [53]:
def txt_vocab(descriptions):
  # To build vocab of all unique words
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab

def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + ' ' + desc)
    
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()

In [54]:
dataset_text = r"D:\Dataset\captions.txt"
dataset_images = r"D:\Dataset\Images"

filename = dataset_text
descriptions = img_capt(filename)
print("Length of descriptions =" ,len(descriptions))
#cleaning the descriptions
clean_descriptions = txt_clean(descriptions)
#to build vocabulary
vocabulary = txt_vocab(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))
#saving all descriptions in one file
save_descriptions(clean_descriptions, "descriptions.txt")

Length of descriptions = 8092
Length of vocabulary =  8763


In [55]:
model = Xception( include_top=False, pooling='avg' )

def extract_features(directory):
    model = Xception( include_top=False, pooling='avg' )
    features = dict() 

    for pic in os.listdir(directory):
        file = directory + "/" + pic
        image = Image.open(file)
        image = image.resize((299, 299))
        image = np.expand_dims(image, axis = 0)
        image = image / 127.5
        image = image - 1.0
        feature = model.predict(image)
        features[pic] = feature
    
    return features

features = extract_features(dataset_images)

Exception ignored in: <function tqdm.__del__ at 0x000001920456DB20>
Traceback (most recent call last):
  File "c:\Python311\Lib\site-packages\tqdm\std.py", line 1149, in __del__
    self.close()
  File "c:\Python311\Lib\site-packages\tqdm\notebook.py", line 278, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x000001920456DB20>
Traceback (most recent call last):
  File "c:\Python311\Lib\site-packages\tqdm\std.py", line 1149, in __del__
    self.close()
  File "c:\Python311\Lib\site-packages\tqdm\notebook.py", line 278, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'
Exception ignored in: <function tqdm.__del__ at 0x000001920456DB20>
Traceback (most recent call last):
  File "c:\Python311\Lib\site-packages\tqdm\std.py", line 1149, in __del__
    self.close()
 

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━

In [56]:
len(features)
dump(features, open("features.p", "wb"))

features  = load(open("features.p", "rb"))

print(features)

{'1000268201_693b08cb0e.jpg': array([[0.4734091 , 0.01730899, 0.07334232, ..., 0.08557966, 0.02102292,
        0.23765537]], dtype=float32), '1001773457_577c3a7d70.jpg': array([[0.00158199, 0.11114135, 0.00037428, ..., 0.265044  , 0.35281494,
        0.05871555]], dtype=float32), '1002674143_1b742ab4b8.jpg': array([[0.        , 0.02493176, 0.01557377, ..., 0.        , 0.        ,
        0.10194052]], dtype=float32), '1003163366_44323f5815.jpg': array([[0.14568   , 0.00272418, 0.2777575 , ..., 0.17016958, 0.11957466,
        0.09415718]], dtype=float32), '1007129816_e794419615.jpg': array([[0.        , 0.12444546, 0.73915863, ..., 0.00390427, 0.00997238,
        0.5017138 ]], dtype=float32), '1007320043_627395c3d8.jpg': array([[0.04138539, 0.        , 0.01274425, ..., 0.00944677, 0.6420151 ,
        0.04793242]], dtype=float32), '1009434119_febe49276a.jpg': array([[0.        , 0.        , 0.02635749, ..., 0.30526015, 0.23072201,
        0.14181001]], dtype=float32), '1012212859_01547e3